In [1]:
from guppylang.decorator import guppy

Test that functions registered inside calls still produce correct diagnostic spans. See https://github.com/CQCL/guppylang/issues/906

In [2]:
def make():    
    @guppy
    def foo() -> int:    
        return 1.0

    return foo

foo = make()

@guppy
def main() -> int:
    return foo()

main.compile();

Error: Type mismatch (at <In[2]>:4:15)
  | 
2 |     @guppy
3 |     def foo() -> int:
4 |         return 1.0
  |                ^^^ Expected return value of type `int`, got `float`

Guppy compilation failed due to 1 previous error


In [3]:
# See https://github.com/CQCL/guppylang/issues/1053

def make():
    @guppy
    def f() -> None:
        1 + False  # Any type error here will do

    return f

make().compile()

Error: Operator not defined (at <In[3]>:6:8)
  | 
4 |     @guppy
5 |     def f() -> None:
6 |         1 + False  # Any type error here will do
  |         ^^^^^^^^^ Binary operator `+` not defined for `int` and `bool`

Guppy compilation failed due to 1 previous error


In [4]:
@guppy.struct
class MyStruct:
    pass

def make():
    @guppy.struct
    class MyStruct:
        x: int

    return MyStruct

MyStruct = make()

@guppy
def main() -> MyStruct:
    return MyStruct()

main.compile()

Error: Not enough arguments (at <In[4]>:16:19)
   | 
14 | @guppy
15 | def main() -> MyStruct:
16 |     return MyStruct()
   |                    ^^ Missing argument (expected 1, got 0)

Note: Function signature is `int -> MyStruct`

Guppy compilation failed due to 1 previous error


In [5]:
# See https://github.com/CQCL/guppylang/issues/937

from guppylang.std.quantum import discard
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit

@guppy.struct
class Struct:
    q: qubit

    @guppy
    def wrong(self: "Struct" @ owned) -> None:
        pass # self.discard() would work

    @guppy
    def discard(self: "Struct" @ owned) -> None:
        discard(self.q)

@guppy
def main() -> None:
    Struct(qubit()).wrong()

    
main.compile()

Error: Drop violation (at <In[5]>:12:14)
   | 
10 | 
11 |     @guppy
12 |     def wrong(self: "Struct" @ owned) -> None:
   |               ^^^^^^^^^^^^^^^^^^^^^^ Field `self.q` with non-droppable type `qubit` is leaked

Help: Make sure that `self.q` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error
